In [1]:
import pandas as pd
import pickle
import sys
import os
sys.path.append("/home/ziniu.wzn/BayesCard")
from DataPrepare.join_data_preparation import JoinDataPreparator
from Models.Bayescard_BN import Bayescard_BN, build_meta_info

In [5]:
hdf_path = "/home/ziniu.wzn/stats/gen_single_light"
meta_data_path = hdf_path + '/meta_data.pkl'
from DataPrepare.schemas import gen_DB0_schema, gen_stats_light_schema
schema = gen_stats_light_schema("/home/ziniu.wzn/stats")
prep = JoinDataPreparator(meta_data_path, schema, max_table_data=20000000)
print(f"BN will be trained on the full outer join of following relations")
for relationship_obj in schema.relationships:
    print(relationship_obj.identifier)

BN will be trained on the full outer join of following relations
comments.PostId = posts.Id
comments.UserId = users.Id
badges.UserId = users.Id
tags.ExcerptPostId = posts.Id
postLinks.PostId = posts.Id
postLinks.RelatedPostId = posts.Id
postHistory.PostId = posts.Id
postHistory.UserId = users.Id
votes.PostId = posts.Id
votes.UserId = users.Id
posts.OwnerUserId = users.Id


In [6]:
algorithm = "chow-liu"
max_parents = 1
sample_size = 200000
model_folder = "/home/ziniu.wzn/stats/BayesCard_temp"

for i, relationship_obj in enumerate(schema.relationships):
    df_sample_size = 10000000
    relation = [relationship_obj.identifier]
    print("training on relation", relation)
    df, meta_types, null_values, full_join_est = prep.generate_n_samples(
        df_sample_size, relationship_list=relation, post_sampling_factor=10)
    columns = list(df.columns)
    assert len(columns) == len(meta_types) == len(null_values)
    
    meta_info = build_meta_info(df.columns, null_values)
    bn = Bayescard_BN(schema, relation, column_names=columns, full_join_size=len(df),
                      table_meta_data=prep.table_meta_data, meta_types=meta_types, null_values=null_values,
                      meta_info=meta_info)
    model_path = model_folder + f"/{i}_{algorithm}_{max_parents}.pkl"
    bn.build_from_data(df, algorithm=algorithm, max_parents=max_parents, ignore_cols=['Id'],
                       sample_size=sample_size)
    pickle.dump(bn, open(model_path, 'wb'), pickle.HIGHEST_PROTOCOL)
    print(f"model saved at {model_path}")

training on relation
Discretizing table takes 1.8333439826965332 secs
Structure learning took 15.725023984909058 secs.
done, parameter learning took 0.9257287979125977 secs.
model saved at /home/ziniu.wzn/stats/BayesCard_temp/0_chow-liu_1.pkl
training on relation


/home/ziniu.wzn/anaconda3/envs/deepdb/lib/python3.7/site-packages/pandas/core/reshape/merge.py:970: UserWarning: You are merging on int and float columns where the float values are not equal to their int representation
  'representation', UserWarning)
/home/ziniu.wzn/BayesCard/DataPrepare/join_data_preparation.py:507: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  df_samples = pd.concat([df_samples, wo_join_partners])


Discretizing table takes 8.109903573989868 secs
Structure learning took 10.367659091949463 secs.
done, parameter learning took 0.7171745300292969 secs.
model saved at /home/ziniu.wzn/stats/BayesCard_temp/1_chow-liu_1.pkl
training on relation
Discretizing table takes 5.9259443283081055 secs
Structure learning took 6.092707872390747 secs.
done, parameter learning took 0.3254077434539795 secs.
model saved at /home/ziniu.wzn/stats/BayesCard_temp/2_chow-liu_1.pkl
training on relation


/home/ziniu.wzn/anaconda3/envs/deepdb/lib/python3.7/site-packages/pandas/core/reshape/merge.py:970: UserWarning: You are merging on int and float columns where the float values are not equal to their int representation
  'representation', UserWarning)
/home/ziniu.wzn/BayesCard/DataPrepare/join_data_preparation.py:507: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  df_samples = pd.concat([df_samples, wo_join_partners])


Discretizing table takes 2.3717033863067627 secs
Structure learning took 9.710107803344727 secs.
done, parameter learning took 0.5084080696105957 secs.
model saved at /home/ziniu.wzn/stats/BayesCard_temp/3_chow-liu_1.pkl
training on relation
Discretizing table takes 1.4450650215148926 secs
Structure learning took 9.705158233642578 secs.
done, parameter learning took 0.48624348640441895 secs.
model saved at /home/ziniu.wzn/stats/BayesCard_temp/4_chow-liu_1.pkl
training on relation
Discretizing table takes 1.4438705444335938 secs
Structure learning took 10.211598634719849 secs.
done, parameter learning took 0.49811863899230957 secs.
model saved at /home/ziniu.wzn/stats/BayesCard_temp/5_chow-liu_1.pkl
training on relation
Discretizing table takes 1.970289707183838 secs
Structure learning took 22.01914095878601 secs.
done, parameter learning took 1.1211318969726562 secs.
model saved at /home/ziniu.wzn/stats/BayesCard_temp/6_chow-liu_1.pkl
training on relation


...

Discretizing table takes 9.526138067245483 secs
Structure learning took 14.238501071929932 secs.
done, parameter learning took 1.066662073135376 secs.
model saved at /home/ziniu.wzn/stats/BayesCard_temp/7_chow-liu_1.pkl
training on relation
Discretizing table takes 6.289434909820557 secs
Structure learning took 20.075660943984985 secs.
done, parameter learning took 1.6056325435638428 secs.
model saved at /home/ziniu.wzn/stats/BayesCard_temp/8_chow-liu_1.pkl
training on relation


/home/ziniu.wzn/anaconda3/envs/deepdb/lib/python3.7/site-packages/pandas/core/reshape/merge.py:970: UserWarning: You are merging on int and float columns where the float values are not equal to their int representation
  'representation', UserWarning)
/home/ziniu.wzn/BayesCard/DataPrepare/join_data_preparation.py:507: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  df_samples = pd.concat([df_samples, wo_join_partners])


Discretizing table takes 15.816088676452637 secs
Structure learning took 7.065018892288208 secs.
done, parameter learning took 1.5047211647033691 secs.
model saved at /home/ziniu.wzn/stats/BayesCard_temp/9_chow-liu_1.pkl
training on relation


/home/ziniu.wzn/anaconda3/envs/deepdb/lib/python3.7/site-packages/pandas/core/reshape/merge.py:970: UserWarning: You are merging on int and float columns where the float values are not equal to their int representation
  'representation', UserWarning)
/home/ziniu.wzn/BayesCard/DataPrepare/join_data_preparation.py:507: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  df_samples = pd.concat([df_samples, wo_join_partners])


Discretizing table takes 8.792738914489746 secs
Structure learning took 20.64398193359375 secs.
done, parameter learning took 1.1227610111236572 secs.
model saved at /home/ziniu.wzn/stats/BayesCard_temp/10_chow-liu_1.pkl
